In [ ]:
#import libraries

import numpy as np
import sqlite3
import itertools
import sqlalchemy

conn = sqlite3.connect("soccer_bundesliga_test.sqlite")
c = conn.cursor()

#set variables
adjperiod = 20
C = 400
midpoint = 2000
initial_score = 2000
initial_string="2000"
initial_home_field_adv = 0.5 *C


#defining model functions
    
    
def getP(ratinga, ratingb, C):
    Pa = 1/(1+np.exp(-1*(ratinga-ratingb)/C))
    return(Pa)


#count number of matches to use for loop later

c.execute('SELECT Count(*) FROM Match')
matchcount = c.fetchall()
matchcount= int(matchcount[0][0])
print(matchcount)

for i in range(matchcount):
    var = i 
    var_1 =i+1
    n_home_team_score_adj = 0
    n_away_team_score_adj = 0
    sum_home_team_score_adj = 0
    sum_away_team_score_adj = 0
    
    print(i)
#get data to calculate

#match info
    c.execute('''SELECT id, home_team_api_id, away_team_api_id, home_field_adv, league_id
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))
    match_info = c.fetchall()
    match_id = int(match_info[0][0])
    home_team_id = int(match_info[0][1])
    away_team_id = int(match_info[0][2])
    home_field_adv = match_info[0][3]
    match_league_id = match_info[0][4]


#check if there is already a home_field_adv written into the db
    if not home_field_adv:
        c.execute('''Update match Set home_field_adv = ? Where ID = ?''',
          (initial_home_field_adv,match_id,))
        home_field_adv = initial_home_field_adv
    c.execute('''SELECT home_team_goal
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))
    g_home = c.fetchall()    
    g_home = int(g_home[0][0])
    c.execute('''SELECT away_team_goal
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))
    g_away = c.fetchall()
    g_away = int(g_away[0][0])
#add game to league gamecounter
    c.execute('''UPDATE League SET games_played = games_played +1 where id = ?''', (match_league_id,))
    
#home team:

    c.execute('''SELECT home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7,
            home_player_8, home_player_9, home_player_10, home_player_11 
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))

    home_team = c.fetchall()
    
    
#away team

    c.execute('''SELECT away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7,
            away_player_8, away_player_9, away_player_10, away_player_11 
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))

    away_team = c.fetchall()
    
    
#get individual scores + game counts for home team

    c.execute('''SELECT home_player_1_score, home_player_2_score, home_player_3_score, home_player_4_score, home_player_5_score, home_player_6_score, 
            home_player_7_score, home_player_8_score, home_player_9_score, home_player_10_score, home_player_11_score 
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))    
    home_team_score= c.fetchall()
    home_team_score =list(itertools.chain.from_iterable(home_team_score))
    
    c.execute('''SELECT home_player_1_gamecount, home_player_2_gamecount, home_player_3_gamecount, home_player_4_gamecount, home_player_5_gamecount, home_player_6_gamecount, 
            home_player_7_gamecount, home_player_8_gamecount, home_player_9_gamecount, home_player_10_gamecount, home_player_11_gamecount 
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))      
    home_team_gamecount= c.fetchall()
    home_team_gamecount =list(itertools.chain.from_iterable(home_team_gamecount))
    
    # add default rating if no rating found yet and write it into db and gamecount +1
    for ii in range(len(home_team_score)):
        
            if home_team_gamecount[ii] == None:
                
                home_team_gamecount[ii] =1
                columncount = 'home_player_'+str(ii+1)+'_gamecount'
                c.execute('''Update match SET %s = 1 Where id = ? ''' % (columncount), (match_id,))
                #conn.commit()
                
                
            if home_team_score[ii] == None:
                
                home_team_score[ii] = initial_score
                columnscore = 'home_player_'+str(ii+1)+'_score'
                c.execute('''Update match SET %s = ? Where id = ? ''' % (columnscore), (initial_score,match_id,))
            
#calculate table of scores where players under x games are weighted lower (adjustment period)

            if home_team_gamecount[ii] > adjperiod:
                sum_home_team_score_adj += home_team_score[ii]
                n_home_team_score_adj += 1
            else:
                sum_home_team_score_adj += home_team_score[ii]*home_team_gamecount[ii]/adjperiod
                n_home_team_score_adj += home_team_gamecount[ii]/adjperiod
                
            
            #conn.commit()
                
            
            
    

#get individual scores for away team

    c.execute('''SELECT away_player_1_score, away_player_2_score, away_player_3_score, away_player_4_score, away_player_5_score, away_player_6_score, 
            away_player_7_score, away_player_8_score, away_player_9_score, away_player_10_score, away_player_11_score 
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,))    
    away_team_score= c.fetchall()
    away_team_score =list(itertools.chain.from_iterable(away_team_score))
    
    
    
    c.execute('''SELECT away_player_1_gamecount, away_player_2_gamecount, away_player_3_gamecount, away_player_4_gamecount, away_player_5_gamecount, away_player_6_gamecount, 
            away_player_7_gamecount, away_player_8_gamecount, away_player_9_gamecount, away_player_10_gamecount, away_player_11_gamecount 
          FROM match ORDER BY DATE ASC LIMIT 1 OFFSET ?  ''', (var,)) 
   
    away_team_gamecount= c.fetchall()
    away_team_gamecount =list(itertools.chain.from_iterable(away_team_gamecount))    

    # add default rating if no rating found yet and write it into db and gamecount +1
    for ii in range(len(away_team_score)):
        
            
            if away_team_gamecount[ii] == None:
                away_team_gamecount[ii] = 1  
                columncount = 'away_player_'+str(ii+1)+'_gamecount'
                c.execute('''Update match SET %s = 1 Where id = ? ''' % (columncount), (match_id,))
                  
            
            if away_team_score[ii] == None:
                away_team_score[ii] = initial_score
                columnscore = 'away_player_'+str(ii+1)+'_score'
                c.execute('''Update match SET %s = ? Where id = ? ''' % (columnscore), (initial_score,match_id,))
                
#calculate table of scores where players under x games are weighted lower (adjustment period)

            if away_team_gamecount[ii] > adjperiod:
                sum_away_team_score_adj += away_team_score[ii]
                n_away_team_score_adj += 1
                
            else:
                sum_away_team_score_adj += away_team_score[ii]*away_team_gamecount[ii]/adjperiod
                n_away_team_score_adj += away_team_gamecount[ii]/adjperiod
                    
            
                
            
    
# calculate team scores    
    if n_home_team_score_adj > 0:    
        home_team_avg = sum_home_team_score_adj/n_home_team_score_adj
    else:
        home_team_avg = initial_score
        
    if n_away_team_score_adj > 0:
        away_team_avg = sum_away_team_score_adj/n_away_team_score_adj
    else:
        away_team_avg = initial_score
#write them into db:

    c.execute('''Update match Set home_team_score = ? Where id = ? ''',
            (home_team_avg, match_id ))
    
    c.execute('''Update match Set away_team_score = ? Where id = ? ''',
            (away_team_avg, match_id ))
    
    
    
    
    
# neural net:

#calculate goal difference to use for determining learning rate later

    g_difference = abs(g_home-g_away)

#determine winner from data and set winner as w1, loser as w2 

    if g_home > g_away:
        w1_home = True
        w2_home = False
        w1 = home_team_avg
        w1_indv = home_team_score
        w1_gamecount = home_team_gamecount
        w1_ids = home_team
        w1_team_id = home_team_id
        
        
        w2 = away_team_avg
        w2_ids = away_team
        w2_indv = away_team_score
        w2_gamecount = away_team_gamecount
        w2_team_id = away_team_id
        
        target_output = 1
    
    elif g_home == g_away:
        w1_home = True
        w2_home = False
        w1 = home_team_avg
        w1_indv = home_team_score
        w1_ids = home_team
        w1_gamecount = home_team_gamecount
        w1_team_id = home_team_id
        
        w2 = away_team_avg
        w2_indv = away_team_score
        w2_ids = away_team
        w2_gamecount = away_team_gamecount
        w2_team_id = away_team_id
        
        target_output = 0.5
    
    else:
        w2_home = True
        w1_home = False
        w1 = away_team_avg
        w1_indv = away_team_score
        w1_ids = away_team
        w1_gamecount = away_team_gamecount
        w1_team_id = away_team_id
        
        w2 = home_team_avg
        w2_indv = home_team_score
        w2_ids = home_team
        w2_gamecount = home_team_gamecount
        w2_team_id = home_team_id
        
        target_output = 1
    


    
#implement team home-field-adv:
    
    if w1_home == True:
        w1 += home_field_adv
    else:
        w2 += home_field_adv

        
#calculate P(w1 beats w2) (Output)

    
    P_w1_beats_w2 = getP(w1,w2,C)

    
    P_w2_beats_w1 = 1 - P_w1_beats_w2

    
#calculate delta error for home-field-adv and calculate new hfa
    learning_rate_hfa = 0.01*C
    if g_home > g_away:
        error_hfa = (1 - P_w1_beats_w2)
        
    elif g_home == g_away:
        error_hfa = (0.5 - P_w1_beats_w2)
        
    else:
        error_hfa =-1*(1 - P_w2_beats_w1)
        
    
    delta_hfa = learning_rate_hfa * error_hfa 
    new_home_field_adv = home_field_adv + delta_hfa
    
    
    
    
#search for next match in league and add new home field adv

    c.execute('''Select games_played from League Where id = ? ''', (match_league_id,))
    league_games_played = c.fetchall()
    league_games_played = int(league_games_played[0][0])
    
    c.execute('''Select ID from Match Where league_id = ? Order by Date asc Limit 1 Offset ?''',
              (match_league_id, league_games_played,))
    id_of_next_match_date = c.fetchall()
    
    if not id_of_next_match_date:
        c.execute('''Update league Set home_field_adv = ? where ID = ?''', (new_home_field_adv, match_league_id,))
    else:
        id_of_next_match_date = int(id_of_next_match_date[0][0])
        c.execute('''Update match Set home_field_adv = ? Where ID = ?''',
          (new_home_field_adv,id_of_next_match_date,))         
        

        
        
#calculate delta erros for individual players    
    for iii in range(len(w1_indv)):

    
        
#get gamecount
        
        w1_gamecount_i = w1_gamecount[iii]
        
#get expected probability of player w1i winning against w2        
        
        
        w1i =w1_indv[iii]
        w1i_calc =w1_indv[iii] 
        if w1_home == True:
            w1i_calc = w1i_calc + home_field_adv
    
            
        
        P_w1i_beats_w2 = getP(w1i_calc,w2,C)      
        output = P_w1i_beats_w2
        
#set learning rate and calculate error + delta to calculate new rating        

        if w1_gamecount_i <= 20:
            learning_rate = 0.15
        elif w1_gamecount_i <= 50:
            learning_rate = 0.1
        else:    
            learning_rate = 0.05
        

        learning_rate = learning_rate * (1+0.2*(g_difference))*C #multiply learning by factor determined by goal difference and C 
        error_w1i = target_output - output
        delta_w1i = learning_rate * error_w1i
        new_rating_w1i = delta_w1i + w1i

        

        
#search for column and row to write new data into
        

        player_id = int(w1_ids[0][iii])
        c.execute('''Select ID FROM match 
            Where ? IN (home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7,
            home_player_8, home_player_9, home_player_10, home_player_11,away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7,
            away_player_8, away_player_9, away_player_10, away_player_11) ORDER BY DATE ASC LIMIT 1 OFFSET ?''', 
                  (player_id, w1_gamecount_i,))
        
        
        ID_of_next_match = c.fetchall()
        new_gamecount = w1_gamecount_i +1
        
        if len(ID_of_next_match) > 0:   #check if there still are matches with player left uncalculated
               
            ID_of_next_match = ID_of_next_match[0][0]




            c.execute('''Select CASE 
                        WHEN home_player_1 = ? Then 'home_player_1'
                        WHEN home_player_2 = ? Then 'home_player_2'
                        WHEN home_player_3 = ? Then 'home_player_3'
                        WHEN home_player_4 = ? Then 'home_player_4'
                        WHEN home_player_5 = ? Then 'home_player_5'
                        WHEN home_player_6 = ? Then 'home_player_6'
                        WHEN home_player_7 = ? Then 'home_player_7'
                        WHEN home_player_8 = ? Then 'home_player_8'
                        WHEN home_player_9 = ? Then 'home_player_9'
                        WHEN home_player_10 = ? Then 'home_player_10'
                        WHEN home_player_11 = ? Then 'home_player_11'
                        WHEN away_player_1 = ? Then 'away_player_1'
                        WHEN away_player_2 = ? Then 'away_player_2'
                        WHEN away_player_3 = ? Then 'away_player_3'
                        WHEN away_player_4 = ? Then 'away_player_4'
                        WHEN away_player_5 = ? Then 'away_player_5'
                        WHEN away_player_6 = ? Then 'away_player_6'
                        WHEN away_player_7 = ? Then 'away_player_7'
                        WHEN away_player_8 = ? Then 'away_player_8'
                        WHEN away_player_9 = ? Then 'away_player_9'
                        WHEN away_player_10 = ? Then 'away_player_10'
                        WHEN away_player_11 = ? Then 'away_player_11'
                        END
                        FROM MATCH WHERE ID = ? ''', (player_id, player_id, player_id, player_id, player_id, 
                        player_id, player_id, player_id, player_id, player_id, player_id,player_id, player_id, player_id, player_id, player_id, 
                        player_id, player_id, player_id, player_id, player_id, player_id,ID_of_next_match,))



            columnplayer = c.fetchall()
            columnplayer = columnplayer[0][0]
            columnscore = columnplayer + '_score'
            columncount = columnplayer + '_gamecount'
            

#write new data into db            
            
            c.execute('''Update match SET %s = ? Where ID = ? 
                       ''' % (columnscore), (new_rating_w1i, ID_of_next_match,  ))
            c.execute('''Update match SET %s = ? Where ID = ? 
                       ''' % (columncount), (new_gamecount, ID_of_next_match,  ))            
            #conn.commit()
        else:
            new_rating_w1i = int(new_rating_w1i) #round to integer
            c.execute('''Update player SET current_score = ? Where player_api_id = ? ''',
                      (new_rating_w1i,player_id,))
           
            c.execute('''Update player SET games_played = ? Where player_api_id = ? ''',
                      (new_gamecount,player_id,))
            
            c.execute('''Update player Set current_team = ? Where player_api_id = ?''',
                     (w1_team_id, player_id,))

    for iii in range(len(w2_indv)):

#get gamecount
        
        w2_gamecount_i = w2_gamecount[iii]         
        

#get expected probability of player w2i winning against w1        
        w2i =w2_indv[iii]
        w2i_calc =w2_indv[iii] 
        if w2_home == True:
            w2i_calc = w2i_calc + home_field_adv

        P_w1_beats_w2i = getP(w1,w2i,C)     
        output = P_w1_beats_w2i
        
#set learning rate and calculate error + delta to calculate new rating        
        
        if w2_gamecount_i <= 20:
            learning_rate = 0.15
        elif w2_gamecount_i <= 50:    
            learning_rate = 0.1
        else: learning_rate = 0.05
        
        
        learning_rate = learning_rate * (1+0.2*(g_difference))*C #multiply learning by factor determined by goal difference and C
        error_w2i =-1*(target_output - output)
        delta_w2i = learning_rate * error_w2i
        new_rating_w2i = delta_w2i + w2i

#search for column and row to write new data into
        
        player_id = int(w2_ids[0][iii])

        c.execute('''Select ID FROM match 
            Where ? IN (home_player_1, home_player_2, home_player_3, home_player_4, home_player_5, home_player_6, home_player_7,
            home_player_8, home_player_9, home_player_10, home_player_11,away_player_1, away_player_2, away_player_3, away_player_4, away_player_5, away_player_6, away_player_7,
            away_player_8, away_player_9, away_player_10, away_player_11) ORDER BY DATE ASC LIMIT 1 OFFSET ?''', 
                  (player_id, w2_gamecount_i,))
        
        ID_of_next_match = c.fetchall()
        new_gamecount = w2_gamecount_i+1
        
        if len(ID_of_next_match)  > 0:        #check if there still are matches with player left uncalculated
        
        

            ID_of_next_match = ID_of_next_match[0][0]
            

            c.execute('''Select CASE 
                        WHEN home_player_1 = ? Then 'home_player_1'
                        WHEN home_player_2 = ? Then 'home_player_2'
                        WHEN home_player_3 = ? Then 'home_player_3'
                        WHEN home_player_4 = ? Then 'home_player_4'
                        WHEN home_player_5 = ? Then 'home_player_5'
                        WHEN home_player_6 = ? Then 'home_player_6'
                        WHEN home_player_7 = ? Then 'home_player_7'
                        WHEN home_player_8 = ? Then 'home_player_8'
                        WHEN home_player_9 = ? Then 'home_player_9'
                        WHEN home_player_10 = ? Then 'home_player_10'
                        WHEN home_player_11 = ? Then 'home_player_11'
                        WHEN away_player_1 = ? Then 'away_player_1'
                        WHEN away_player_2 = ? Then 'away_player_2'
                        WHEN away_player_3 = ? Then 'away_player_3'
                        WHEN away_player_4 = ? Then 'away_player_4'
                        WHEN away_player_5 = ? Then 'away_player_5'
                        WHEN away_player_6 = ? Then 'away_player_6'
                        WHEN away_player_7 = ? Then 'away_player_7'
                        WHEN away_player_8 = ? Then 'away_player_8'
                        WHEN away_player_9 = ? Then 'away_player_9'
                        WHEN away_player_10 = ? Then 'away_player_10'
                        WHEN away_player_11 = ? Then 'away_player_11'
                        END
                        FROM MATCH WHERE ID = ? ''', (player_id, player_id, player_id, player_id, player_id, 
                        player_id, player_id, player_id, player_id, player_id, player_id,player_id, player_id, player_id, player_id, player_id, 
                        player_id, player_id, player_id, player_id, player_id, player_id,ID_of_next_match,))


            columnplayer = c.fetchall()[0][0]
            columnscore = columnplayer + '_score'
            columncount = columnplayer + '_gamecount'
            


            #write new data into db
            c.execute('''Update match SET %s = ? Where ID = ? 
                       ''' % (columnscore), (new_rating_w2i, ID_of_next_match,  ))
            c.execute('''Update match SET %s = ? Where ID = ? 
                       ''' % (columncount), (new_gamecount, ID_of_next_match,  ))  

            
        else:
            new_rating_w2i = int(new_rating_w2i) #round to integer
            c.execute('''Update player SET current_score = ? Where player_api_id = ? ''',
                      (new_rating_w2i,player_id,))
            c.execute('''Update player SET games_played = ? Where player_api_id = ? ''',
                      (new_gamecount,player_id,))
            c.execute('''Update player Set current_team = ? Where player_api_id = ?''',
                     (w2_team_id, player_id,))
        
        


    conn.commit()

print('finished')